<div style="text-align:center"><span style="font-size:2em; font-weight: bold;">Lecture 9—Clustering</span></div>

# $k$-Means clustering

In [69]:
import numpy as np
import pandas as pd
from cleands import *

Data Generating Process

In [149]:
x1 = np.random.normal(loc=np.random.uniform(size=(5,))*10-5,size=(500,5))
x2 = np.random.normal(loc=np.random.uniform(size=(5,))*10-5,size=(500,5))
x3 = np.random.normal(loc=np.random.uniform(size=(5,))*10-5,size=(500,5))
x = np.vstack((x1,x2,x3))
shuffle = np.random.permutation(x.shape[0])
x = x[shuffle,:]
membership = shuffle.copy()
for i in range(len(membership)):
    if membership[i]<500: membership[i]=0
    elif membership[i]<1000: membership[i]=1
    else: membership[i]=2
np.unique(membership,return_counts=True)
membership

array([0, 1, 2, ..., 2, 2, 0])

Calculate means of membership variable

In [150]:
means = []
for i in range(3):
    mean = x[membership==i,:].mean(0)
    means += [mean]
means = np.array(means)
means

array([[-4.81833234, -2.77189149,  0.39174824, -2.47186509, -3.52482939],
       [-0.77038751, -1.32949694, -3.44153593, -4.08948222, -4.10212103],
       [ 0.06807657, -0.28013588,  0.90856008,  1.18996657, -3.04146182]])

In [151]:
x

array([[-3.98683834, -2.41722155, -1.31090168, -3.67670059, -4.19493151],
       [-3.09589967, -2.36298148, -3.04965523, -5.09462767, -6.02356359],
       [-1.0823621 ,  0.15525515,  0.55498562,  1.32264453, -3.02067485],
       ...,
       [-0.7611711 , -0.12308007,  0.73690964, -0.69205775, -4.78088394],
       [-0.53584459,  3.34895359,  2.05515834,  1.75209228, -1.38124049],
       [-4.86636001, -1.76432631,  1.45508667, -2.3073294 , -3.88196045]])

kmeans step 1: randomly guess

In [152]:
k = 3
n = x.shape[0]
group = np.random.randint(k,size=(n,))
group

array([0, 1, 0, ..., 1, 0, 1])

kmeans step 2: calculate means of each cluster

In [153]:
means = []
for i in range(k):
    mean = x[group==i,:].mean(0)
    means += [mean]
means = np.array(means)
means

array([[-1.8771637 , -1.47233659, -0.66956903, -1.73528266, -3.54725348],
       [-1.84480543, -1.4733826 , -0.77030822, -1.86827674, -3.53685397],
       [-1.8021401 , -1.43790452, -0.7025926 , -1.76999444, -3.5818936 ]])

kmeans step 3: group each point to its closest mean

In [154]:
dists = []
for i in range(k):
    dist = x-means[i,:]
    dist = (dist**2).sum(1)
    dists += [dist]
dists = np.array(dists)
group = dists.argmin(0)
group

array([1, 1, 0, ..., 2, 0, 0], dtype=int64)

kmeans step 4: go back to step 2 until converges...

Putting it all together:

In [155]:
k = 3
n = x.shape[0]
max_iters = 100
newgroup = np.random.randint(k,size=(n,))
group = np.zeros((n,))
for j in range(max_iters):
    if (group==newgroup).all(): break
    print('iteration')
    group = newgroup
    dists = []
    for i in range(k):
        mean = x[group==i,:].mean(0)
        dist = x-mean
        dist = (dist**2).sum(1)
        dists += [dist]
    dists = np.array(dists)
    newgroup = dists.argmin(0)
group

iteration
iteration
iteration
iteration


array([2, 0, 1, ..., 1, 1, 2], dtype=int64)

Confusion matrix

In [156]:
membershipohe = np.zeros((membership.size, membership.max()+1))
membershipohe[np.arange(membership.size),membership] = 1
groupohe = np.zeros((group.size, group.max()+1))
groupohe[np.arange(group.size),group] = 1
membershipohe.T@groupohe

array([[  0.,   1., 499.],
       [498.,   0.,   2.],
       [  0., 500.,   0.]])

accuracy

In [157]:
(membershipohe.T@groupohe).max(1).sum()/groupohe.sum()

0.998

putting all this in a function

In [158]:
def kmeans(x,k,max_iters=100,seed=None):
    n = x.shape[0]
    if seed != None: np.random.seed(seed)
    newgroup = np.random.randint(k,size=(n,))
    group = np.zeros((n,))
    for j in range(max_iters):
        if (group==newgroup).all(): break
        #print('iteration')
        group = newgroup
        dists = []
        for i in range(k):
            mean = x[group==i,:].mean(0)
            dist = x-mean
            dist = (dist**2).sum(1)
            dists += [dist]
        dists = np.array(dists)
        newgroup = dists.argmin(0)
    return newgroup

Total within sum of squares calculation

In [159]:
k = 5
group = kmeans(x,k)
means = np.array([x[group==i,:].mean(0) for i in range(k)])
wss = [((x[group==i,:]-means[i,:])**2).sum() for i in range(k)]
total_wss = sum(wss)
total_wss

7080.098891171296

Loop process and get min twss

In [160]:
def rep_kmeans(x,k,max_iters=100,seed=None,n_start=100):
    twss = []
    groups = []
    for i in range(n_start):
        group = kmeans(x,k,max_iters,seed)
        means = np.array([x[group==i,:].mean(0) for i in range(k)])
        wss = [((x[group==i,:]-means[i,:])**2).sum() for i in range(k)]
        total_wss = sum(wss)
        groups += [group]
        twss += [total_wss]
    group = groups[np.array(twss).argmin()]
    return group

In [161]:
k = 10
group = rep_kmeans(x,k)
means = np.array([x[group==i,:].mean(0) for i in range(k)])
wss = [((x[group==i,:]-means[i,:])**2).sum() for i in range(k)]
total_wss = sum(wss)
total_wss

C:\Users\jason\AppData\Local\Temp\ipykernel_17248\2381808098.py:12: RuntimeWarning: Mean of empty slice.
  mean = x[group==i,:].mean(0)
C:\Users\jason\AppData\Local\Temp\ipykernel_17248\253577756.py:6: RuntimeWarning: Mean of empty slice.
  means = np.array([x[group==i,:].mean(0) for i in range(k)])


5750.20009337508

Automatic elbow detector

In [162]:
def auto_kmeans(x,k_max=10,max_iters=100,seed=None,n_start=100):
    groups = []
    twss = []
    for k in range(1,k_max):
        group = rep_kmeans(x,k,max_iters,seed,n_start)
        means = np.array([x[group==i,:].mean(0) for i in range(k)])
        wss = [((x[group==i,:]-means[i,:])**2).sum() for i in range(k)]
        total_wss = sum(wss)
        groups += [group]
        twss += [total_wss]
    twss = np.array(twss)
    dwss = -np.diff(twss)
    dwss = np.insert(dwss,0,dwss.sum()/np.log(k_max))
    dwss = np.trim_zeros(dwss)
    ratio = dwss[:-1]/dwss[1:]
    ratio = ratio[:k_max]
    k = ratio.argmax()
    return groups[k]

In [163]:
result = auto_kmeans(x)
np.unique(result)

C:\Users\jason\AppData\Local\Temp\ipykernel_17248\2381808098.py:12: RuntimeWarning: Mean of empty slice.
  mean = x[group==i,:].mean(0)
C:\Users\jason\AppData\Local\Temp\ipykernel_17248\253577756.py:6: RuntimeWarning: Mean of empty slice.
  means = np.array([x[group==i,:].mean(0) for i in range(k)])


array([0, 1, 2], dtype=int64)

In [164]:
membershipohe = np.zeros((membership.size, membership.max()+1))
membershipohe[np.arange(membership.size),membership] = 1
resultohe = np.zeros((result.size, result.max()+1))
resultohe[np.arange(result.size),result] = 1
membershipohe.T@resultohe

array([[499.,   1.,   0.],
       [  2.,   0., 498.],
       [  0., 500.,   0.]])

In [165]:
(membershipohe.T@resultohe).max(1).sum()/membershipohe.sum()

0.998

# Programming challenges

## Quick sort

Write a program which implements the quick sort algorithm.



## $k$-Means class structure

Write a class structure for our k-means code
